In [5]:
import pandas as pd
import socket
import struct
import random
import platform
from typing import Optional

IS_WINDOWS = (platform.system() == "Windows")


def checksum(data: bytes) -> int:
    if len(data) % 2:
        data += b"\0"
    res = sum(struct.unpack("!%dH" % (len(data) // 2), data))
    while res >> 16:
        res = (res & 0xFFFF) + (res >> 16)
    return (~res) & 0xFFFF


def build_ip_header(src_ip: str, dst_ip: str, payload_len: int, proto: int = socket.IPPROTO_TCP) -> bytes:
    version_ihl = (4 << 4) + 5
    tos = 0
    total_length = 20 + payload_len
    identification = random.randint(0, 65535)
    flags_fragment = 0
    ttl = 64
    header_checksum = 0
    src = socket.inet_aton(src_ip)
    dst = socket.inet_aton(dst_ip)

    ip_header = struct.pack(
        "!BBHHHBBH4s4s",
        version_ihl, tos, total_length, identification,
        flags_fragment, ttl, proto, header_checksum,
        src, dst
    )
    chksum = checksum(ip_header)
    ip_header = struct.pack(
        "!BBHHHBBH4s4s",
        version_ihl, tos, total_length, identification,
        flags_fragment, ttl, proto, chksum,
        src, dst
    )
    return ip_header


def build_tcp_header(
    src_ip: str,
    dst_ip: str,
    src_port: int,
    dst_port: int,
    payload: bytes = b"",
    seq: Optional[int] = None,
    ack_seq: int = 0,
    flags: int = 0x02,
    window: int = 65535
) -> bytes:
    if seq is None:
        seq = random.randint(0, 0xFFFFFFFF)

    doff_reserved = (5 << 4)
    checksum_tcp = 0
    urg_ptr = 0

    tcp_header = struct.pack(
        "!HHLLBBHHH",
        src_port, dst_port, seq, ack_seq,
        doff_reserved, flags, window,
        checksum_tcp, urg_ptr
    )

    placeholder = 0
    protocol = socket.IPPROTO_TCP
    tcp_length = len(tcp_header) + len(payload)

    pseudo_header = struct.pack(
        "!4s4sBBH",
        socket.inet_aton(src_ip),
        socket.inet_aton(dst_ip),
        placeholder, protocol,
        tcp_length
    )

    chksum = checksum(pseudo_header + tcp_header + payload)

    tcp_header = struct.pack(
        "!HHLLBBHHH",
        src_port, dst_port, seq, ack_seq,
        doff_reserved, flags, window,
        chksum, urg_ptr
    )
    return tcp_header


class RawTcpTransport:
    """
    macOS:
    - tries raw socket (requires sudo) and falls back to normal TCP socket (no sudo)
    """
    def __init__(self, dst_ip: str, dst_port: int, src_ip: str = "127.0.0.1", src_port: int = 0):
        self.src_ip = src_ip
        self.dst_ip = dst_ip
        self.src_port = src_port if src_port else random.randint(1024, 65535)
        self.dst_port = dst_port

        self.mode = "tcp"  # default on macOS without sudo
        self.sock = None

        # On macOS, raw socket will fail without sudo -> catch and fallback
        try:
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_RAW, socket.IPPROTO_RAW)
            self.mode = "raw"
        except PermissionError:
            self.mode = "tcp"
        except OSError:
            self.mode = "tcp"

    def encapsulate(self, data: bytes, flags: int = 0x02) -> bytes:
        tcp = build_tcp_header(self.src_ip, self.dst_ip, self.src_port, self.dst_port, payload=data, flags=flags)
        ip = build_ip_header(self.src_ip, self.dst_ip, len(tcp) + len(data))
        return ip + tcp + data

    def send(self, data: bytes, flags: int = 0x18):
        if self.mode == "raw":
            pkt = self.encapsulate(data, flags=flags)
            self.sock.sendto(pkt, (self.dst_ip, 0))
            print(f"--> SENT RAW IPv4+TCP ({len(pkt)} bytes)")
        else:
            # Normal TCP (needs a listening server!)
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                s.connect((self.dst_ip, self.dst_port))
                s.sendall(data)
            print(f"--> SENT NORMAL TCP ({len(data)} bytes) to {self.dst_ip}:{self.dst_port}")


# IMPORTANT: match your server.py port
transport = RawTcpTransport(dst_ip="127.0.0.1", dst_port=55556)
print(f"Transport ready. mode={transport.mode}, dst=127.0.0.1:{transport.dst_port}")


OSError: [Errno 48] Address already in use

In [6]:
import pandas as pd
import time
import socket

filename = "./group05_chat_input.csv"
PORT = 55556
HOST = "127.0.0.1"

messages_df = pd.read_csv(filename)
messages_df["message"] = messages_df["message"].fillna("").astype(str)

required_cols = ["msg_id", "app_protocol", "src_app", "dst_app", "message", "timestamp"]
missing = [c for c in required_cols if c not in messages_df.columns]
if missing:
    raise ValueError(f"CSV is missing required columns: {missing}")

def chat_session(username: str, target: str, messages: list[str], delay: float = 0.5):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT))

        # Server asks ENTER_NAME
        s.recv(1024)
        s.sendall(username.encode("utf-8"))

        # Server asks WHO_TO_CONNECT
        s.recv(1024)
        s.sendall(target.encode("utf-8"))

        # Optional "CONNECTED..." message
        try:
            s.recv(1024)
        except:
            pass

        for m in messages:
            s.sendall(m.encode("utf-8"))
            time.sleep(delay)

# Group messages by (src_app -> dst_app)
grouped = {}
for _, row in messages_df.iterrows():
    src = str(row["src_app"])
    dst = str(row["dst_app"])
    msg = str(row["message"])
    grouped.setdefault((src, dst), []).append(msg)

print(f"Prepared {sum(len(v) for v in grouped.values())} messages in {len(grouped)} sessions.")

for (src, dst), msgs in grouped.items():
    print(f"\n[SESSION] {src} -> {dst} ({len(msgs)} msgs)")
    chat_session(src, dst, msgs, delay=0.5)

print("\nDONE.")


Prepared 3 messages in 2 sessions.

[SESSION] client_A -> client_B (2 msgs)

[SESSION] client_B -> client_A (1 msgs)

DONE.
